In [ ]:
import torch
import torchvision
from torchvision import datasets,transforms 
import torch.nn as nn
from torch.optim import lr_scheduler 
import torch.optim as optim
from torchvision import models
import zipfile
import numpy as np
import copy
import matplotlib.pyplot as plt
import time
import os

In [ ]:
with zipfile.ZipFile("face_data.zip","r") as zip_ref:
    zip_ref.extractall("./")
with zipfile.ZipFile("face_test.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [ ]:
mean_arr = [0.485, 0.456, 0.406]
std_arr = [0.229, 0.224, 0.225]

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std)
])

In [ ]:
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std)
])

In [ ]:
train_path = 'face_data/train/'
test_path = 'face_data/val/'

train_data = datasets.ImageFolder(root=train_path,
                                  transform=train_transform)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=4,
                                           shuffle=True, num_workers=4)

test_data = datasets.ImageFolder(root=test_path,
                                transform=test_transform)

test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=8,
                                          shuffle=True, num_workers=4)

data_loader_dict = {
    'train': train_data_loader,
    'test': test_data_loader
}

class_names = train_data.classes

In [ ]:
def imshow(input, title):
    input = input.cpu().numpy().transpose((1, 2, 0))
    input = std * input + mean
    input = np.clip(input, 0, 1)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(input)
    plt.title(title)
    plt.pause(2)

In [ ]:
inputs, classes = next(iter(dataloaders['train']))
output = torchvision.utils.make_grid(inputs)
imshow(output, title=[class_names[x] for x in classes])

model = models.alexnet(pretrained=True)
num_of_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_of_features, len(class_names))
entropy_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

dataset_length_dict = {
    'train': len(train_data),
    'test': len(test_data)
}

In [ ]:
def train_model(model, loss, optimizer, num_of_epochs=500):
    c_time = time.time()
    loss_arr = list()
    best_model = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    for epoch in range(num_of_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_of_epochs))
        for stage in ['train', 'test']:
            if stage == 'train':
                model.train()
            else:
                model.eval()
            current_loss = 0.0
            current_acc = 0
            for inputs, labels in dataloaders[stage]:
                optimizer.zero_grad()
                with torch.set_grad_enabled(stage == 'train'):
                    outputs = model(inputs)
                    _, predictions = torch.max(outputs, 1)
                    loss = loss(outputs, labels)
                    if stage == 'train':
                        loss.backward()
                        optimizer.step()
                
                loss_arr.append(loss.item())
                current_loss += loss.item() * inputs.size(0)
                current_acc += torch.sum(predictions == labels.data)
            epoch_loss = current_loss / dataset_length_dict[stage]
            epoch_acc = current_acc.double()/dataset_length_dict[stage]

            if stage == 'test' and epoch_acc > best_model:
                best_model = epoch_acc
                best_model = copy.deepcopy(model.state_dict())
    print('Best accuracy: {:4f}'.format(best_model))
    model.load_state_dict(best_model)
    return model, loss_arr

In [ ]:
model, loss_arr = train_model(model, entropy_loss, optimizer, num_of_epochs=500)

In [ ]:
with torch.no_grad():
    acc = 0
    total = 0
    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean,
                             std=std)
    ])
    path = "face_data/val/"

    test_data = datasets.ImageFolder(root=path,
                                transform=test_transform)

    test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=8,
                                          shuffle=True, num_workers=4)

    for images, labels in test_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        acc += (predicted == labels).sum().item()

In [ ]:
torch.save(model, './model_final')
file_name = './model_final'
s3 = boto3.resource('s3')
bucket = 'attendance-model'
s3.meta.client.upload_file(file_name, bucket, 'model_final')